**Short version**:

Suppose you have two tensors, where `y_hat` contains computed scores for each class (for example, from y = W*x +b) and `y_true` contains one-hot encoded true labels. 

In [ ]:
y_hat  = ... # Predicted label, e.g. y = tf.matmul(X, W) + b
y_true = ... # True label, one-hot encoded

If you interpret the scores in `y_hat` as unnormalized log probabilities, then they are **logits**.

Additionally, the total cross-entropy loss computed in this manner

In [ ]:
y_hat_softmax = tf.nn.softmax(y_hat)
total_loss = tf.reduce_mean(-tf.reduce_sum(y_true * tf.log(y_hat_softmax), [1]))

is essentially equivalent to the total cross-entropy loss computed with the function `softmax_cross_entropy_with_logits()`:

In [ ]:
total_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_hat, y_true))

**Long version:**

In the output layer of your neural network, you will probably compute an array that contains the class scores for each of your training instances, such as from a computation `y_hat = W*x + b`. To serve as an example, below I've created a `y_hat` as a 2 x 3 array, where the rows correspond to the training instances and the columns correspond to classes. So here there are 2 training instances and 3 classes.

In [ ]:
import tensorflow as tf
import numpy as np

sess = tf.Session()

# Create example y_hat.
y_hat = tf.convert_to_tensor(np.array([[0.5, 1.5, 0.1],[2.2, 1.3, 1.7]]))
sess.run(y_hat)
# array([[ 0.5,  1.5,  0.1],
#        [ 2.2,  1.3,  1.7]])

Note that the values are not normalized (i.e. the rows don't add up to 1). In order to normalize them, we can apply the softmax function, which interprets the input as unnormalized log probabilities (aka **logits**) and outputs normalized linear probabilities. 

In [ ]:
y_hat_softmax = tf.nn.softmax(y_hat)
sess.run(y_hat_softmax)
# array([[ 0.227863  ,  0.61939586,  0.15274114],
#        [ 0.49674623,  0.20196195,  0.30129182]])

It's important to fully understand what the softmax output is saying. Below I've shown a table that more clearly represents the output above. It can be seen that, for example, the probability of training instance 1 being "Class 2" is 0.619. The class probabilities for each training instance are normalized, so the sum of each row is 1.0.

                          Pr(Class 1)  Pr(Class 2)  Pr(Class 3)
                        ,--------------------------------------
    Training instance 1 | 0.227863   | 0.61939586 | 0.15274114
    Training instance 2 | 0.49674623 | 0.20196195 | 0.30129182
    
    
So now we have class probabilities for each training instance, where we can take the argmax() of each row to generate a final classification. From above, we may generate that training instance 1 belongs to "Class 2" and training instance 2 belongs to "Class 1". 

Are these classifications correct? We need to measure against the true labels from the training set. You will need a one-hot encoded `y_true` array, where again the rows are training instances and columns are classes. Below I've created an example `y_true` one-hot array where the true label for training instance 1 is "Class 2" and the true label for training instance 2 is "Class 3".

In [ ]:
y_true = tf.convert_to_tensor(np.array([[0.0, 1.0, 0.0],[0.0, 0.0, 1.0]]))
sess.run(y_true)
# array([[ 0.,  1.,  0.],
#        [ 0.,  0.,  1.]])

Is the probability distribution in `y_hat_softmax` close to the probability distribution in `y_true`? We can use cross-entropy loss to measure the error.

\begin{equation}
H(p, q) = - \sum_{x} p(x) \log [q(x)]
\end{equation}

We can compute the cross-entropy loss on a row-wise basis and see the results. Below we can see that training instance 1 has a loss of 0.479, while training instance 2 has a higher loss of 1.200. This result makes sense because in our example above, `y_hat_softmax` showed that training instance 1's highest probability was for "Class 2", which matches training instance 1 in `y_true`; however, the prediction for training instance 2 showed a highest probability for "Class 1", which does not match the true class "Class 3".

In [ ]:
loss_per_instance_1 = -tf.reduce_sum(y_true * tf.log(y_hat_softmax), reduction_indices=[1])
sess.run(loss_per_instance_1)
# array([ 0.4790107 ,  1.19967598])

What we really want is the total loss over all the training instances. So we can compute:

In [ ]:
total_loss_1 = tf.reduce_mean(-tf.reduce_sum(y_true * tf.log(y_hat_softmax), reduction_indices=[1]))
sess.run(total_loss_1)
# 0.83934333897877944

**Using softmax_cross_entropy_with_logits()**

We can instead compute the total cross entropy loss using the `tf.nn.softmax_cross_entropy_with_logits()` function, as shown below. 

In [ ]:
loss_per_instance_2 = tf.nn.softmax_cross_entropy_with_logits(y_hat, y_true)
sess.run(loss_per_instance_2)
# array([ 0.4790107 ,  1.19967598])

total_loss_2 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_hat, y_true))
sess.run(total_loss_2)
# 0.83934333897877922

Note that `total_loss_1` and `total_loss_2` produce essentially equivalent results with some small differences in the very final digits. However, you might as well use the second approach: it takes one less line of code and accumulates less numerical error because the softmax is done for you inside of `softmax_cross_entropy_with_logits()`.

# Sigmoid functions family

1. tf.nn.sigmoid_cross_entropy_with_logits
2. tf.nn.weighted_cross_entropy_with_logits
3. tf.losses.sigmoid_cross_entropy
4. tf.contrib.losses.sigmoid_cross_entropy (DEPRECATED)

As stated earlier, sigmoid loss function is for binary classification. But tensorflow functions are more general and allow to do multi-label classification, when the classes are independent. In other words, tf.nn.sigmoid_cross_entropy_with_logits solves N binary classifications at once.

The labels must be one-hot encoded or can contain soft class probabilities.

tf.losses.sigmoid_cross_entropy in addition allows to set the in-batch weights, i.e. make some examples more important than others. tf.nn.weighted_cross_entropy_with_logits allows to set class weights (remember, the classification is binary), i.e. make positive errors larger than negative errors. This is useful when the training data is unbalanced.

# Softmax functions family

1. tf.nn.softmax_cross_entropy_with_logits (DEPRECATED IN 1.5)
2. tf.nn.softmax_cross_entropy_with_logits_v2
3. tf.losses.softmax_cross_entropy
4. tf.contrib.losses.softmax_cross_entropy (DEPRECATED)

For softmax_cross_entropy_with_logits, labels must have the shape [batch_size, num_classes] and dtype float32 or float64.
Labels used in softmax_cross_entropy_with_logits are the one hot version of labels used in sparse_softmax_cross_entropy_with_logits. So, to make it work using softmax_cross_entropy_with_logits, you will have to do tf.one_hot on the labels.

These loss functions should be used for multinomial mutually exclusive classification, i.e. pick one out of N classes. Also applicable when N = 2.

The labels must be one-hot encoded or can contain soft class probabilities: a particular example can belong to class A with 50% probability and class B with 50% probability. Note that strictly speaking it doesn't mean that it belongs to both classes, but one can interpret the probabilities this way.

Just like in sigmoid family, tf.losses.softmax_cross_entropy allows to set the in-batch weights, i.e. make some examples more important than others. As far as I know, as of tensorflow 1.3, there's no built-in way to set class weights.

In tensorflow 1.5, v2 version was introduced and the original softmax_cross_entropy_with_logits loss got deprecated. The only difference between them is that in a newer version, backpropagation happens into both logits and labels.

In supervised learning one doesn't need to backpropagate to labels. They are considered fixed ground truth and only the weights need to be adjusted to match them.

But in some cases, the labels themselves may come from a differentiable source, another network. One example might be adversarial learning. In this case, both networks might benefit from the error signal. That's the reason why tf.nn.softmax_cross_entropy_with_logits_v2 was introduced. Note that when the labels are the placeholders (which is also typical), there is no difference if the gradient through flows or not, because there are no variables to apply gradient to.

# Sparse functions family

1. tf.nn.sparse_softmax_cross_entropy_with_logits
2. tf.losses.sparse_softmax_cross_entropy
3. tf.contrib.losses.sparse_softmax_cross_entropy (DEPRECATED)

For sparse_softmax_cross_entropy_with_logits labels must have the shape [batch_size] and the dtype int32 or int64. Each label is an int in range [0, num_classes-1].

Like ordinary softmax above, these loss functions should be used for multinomial mutually exclusive classification, i.e. pick one out of N classes. The difference is in labels encoding: the classes are specified as integers (class index), not one-hot vectors. Obviously, this doesn't allow soft classes, but it can save some memory when there are thousands or millions of classes. However, note that logits argument must still contain logits per each class, thus it consumes at least [batch_size, classes] memory.

Like above, tf.losses version has a weights argument which allows to set the in-batch weights

# Sampled softmax functions family

1. tf.nn.sampled_softmax_loss
2. tf.contrib.nn.rank_sampled_softmax_loss
3. tf.nn.nce_loss

These functions provide another alternative for dealing with huge number of classes. Instead of computing and comparing an exact probability distribution, they compute a loss estimate from a random sample.

The arguments weights and biases specify a separate fully-connected layer that is used to compute the logits for a chosen sample.

Like above, labels are not one-hot encoded, but have the shape [batch_size, num_true].

Sampled functions are only suitable for training. In test time, it's recommended to use a standard softmax loss (either sparse or one-hot) to get an actual distribution.

Another alternative loss is tf.nn.nce_loss, which performs noise-contrastive estimation. I've included this function to the softmax family, because NCE guarantees approximation to softmax in the limit.

# Multi-class cross-entropy

# Binary cross-entropy